In [1]:
#Q1. Install MLflow
import mlflow
print(mlflow.__version__)

2.22.0


In [2]:
#Q2. Download and preprocess the data
!wget -P data https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet 

--2025-05-14 16:21:38--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.164.82.112, 3.164.82.197, 3.164.82.160, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.164.82.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1533740 (1.5M) [binary/octet-stream]
Saving to: ‘data/green_tripdata_2023-02.parquet.1’

green_tripdata_2023 100%[===================>]   1.46M  2.40MB/s    in 0.6s    

2025-05-14 16:21:39 (2.40 MB/s) - ‘data/green_tripdata_2023-02.parquet.1’ saved [1533740/1533740]



In [3]:
!wget -P data https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet 

--2025-05-14 16:21:40--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.164.82.112, 3.164.82.197, 3.164.82.160, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.164.82.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1427002 (1.4M) [binary/octet-stream]
Saving to: ‘data/green_tripdata_2023-01.parquet.1’

green_tripdata_2023 100%[===================>]   1.36M  2.25MB/s    in 0.6s    

2025-05-14 16:21:41 (2.25 MB/s) - ‘data/green_tripdata_2023-01.parquet.1’ saved [1427002/1427002]



In [4]:
!wget -P data https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-03.parquet

--2025-05-14 16:21:41--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-03.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.164.82.197, 3.164.82.40, 3.164.82.112, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.164.82.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1730999 (1.7M) [binary/octet-stream]
Saving to: ‘data/green_tripdata_2023-03.parquet.1’

green_tripdata_2023 100%[===================>]   1.65M  2.67MB/s    in 0.6s    

2025-05-14 16:21:42 (2.67 MB/s) - ‘data/green_tripdata_2023-03.parquet.1’ saved [1730999/1730999]



In [5]:
!python preprocess_data.py --raw_data_path data --dest_path ./output


In [ ]:
####Q4. Launch the tracking server locally

<h1>4 files in output folder<h1>

Validation RMSE: 5.431162180141208


In [7]:
!pip install --force-reinstall --no-cache-dir scipy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 64.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 89.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy━━━━━━━━━━━━━━━━━━━ 0/2 [numpy]
    Found existing installation: scipy 1.13.1 0/2 [numpy]
    Uninstalling scipy-1.13.1:0m╺━━━━━━━━━━━━━━━━━━━ 1/2 [scipy]
      Successfully uninstalled scipy-1.13.1━━━━━━━━━━━━━━━━━━━ 1/2 [scipy]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [scipy]32m1/2 [scipy]


In [23]:
!python train.py --data_path output/

In [24]:
!python hpo.py

100%|██████████| 15/15 [01:01<00:00,  4.12s/trial, best loss: 5.335419588556921]


In [ ]:
#Q5. Tune model hyperparameters
4.817
5.335
5.818
6.336



##Answer:5.335


In [27]:
#Q5 

!python register_model.py 

Traceback (most recent call last):
  File "/workspaces/mlops-zoom-camp/03-training/experiment_tracking/register_model.py", line 102, in <module>
    run_register_model()
  File "/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/click/core.py", line 1161, in __call__
    return self.main(*args, **kwargs)
  File "/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/click/core.py", line 1082, in main
    rv = self.invoke(ctx)
  File "/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/click/core.py", line 1443, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/click/core.py", line 788, in invoke
    return __callback(*args, **kwargs)
  File "/workspaces/mlops-zoom-camp/03-training/experiment_tracking/register_model.py", line 94, in run_register_model
    print(f"Registering model from run_id={run_id} with test_rmse={best_ru

<h1>5.060 <h1>


server cras